In [1]:
!pip install -qU crewai crewai_tools langchain-google-genai python-dotenv serpapi

In [4]:
import os
os.environ["SERPAPI_API_KEY"] = ""
from crewai import LLM, Agent, Task, Crew, Process
from crewai_tools import SerpApiGoogleSearchTool
os.environ["GOOGLE_API_KEY"] = ""
my_llm = LLM(model="gemini/gemini-2.5-flash", temperature=0.2)
print("✅ CrewAI Installed.")

✅ CrewAI Installed.


In [9]:
search_tool = SerpApiGoogleSearchTool()

# 1. Agents
researcher = Agent(
    role="Researcher",
    goal="Fetch latest market news and current stock price",
    tools=[search_tool],
    llm=my_llm,
    verbose=True,
    backstory = "You are an expert in Stock Market Research"
)

analyst = Agent(
    role="Analyst",
    goal="Extract the top 3 key insights from raw research data",
    llm=my_llm,
    backstory="You are a strategic analyst who filters noise and finds patterns.",
    verbose=True
)

writer = Agent(
    role="Executive Writer",
    goal="Write a concise, professional executive briefing",
    llm=my_llm,
    backstory="You write clear, decision-ready summaries for senior leadership.",
    verbose=True
)

# 1. Tasks
research_task = Task(
    description=(
        "Search the web for recent articles, reports, and discussions about {topic}. "
        "Return only raw facts, headlines, and data."
    ),
    expected_output="Bullet points of raw findings with sources.",
    agent=researcher
)

analysis_task = Task(
    description=(
        "Analyze the research data and identify the top 3 key trends or insights."
    ),
    expected_output="Three concise insights with brief explanations.",
    agent=analyst,
    context = [research_task]
)

writing_task = Task(
    description=(
        "Convert the insights into a one-page executive summary in markdown format."
    ),
    expected_output="A polished markdown briefing ready for leadership review.",
    agent=writer,
    context = [analysis_task]
)


In [10]:
crew = Crew(
    agents=[researcher, analyst, writer],
    tasks=[research_task, analysis_task, writing_task],
    process=Process.sequential
)

# 🟢 DEMO TIME: Run this cell to show the linear handoff.
# result = pipeline_crew.kickoff()
# print("Pipeline Result:", result)

result = crew.kickoff(
    inputs={"topic": "AI Trends in 2025"}
)

print(result)

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Researcher                                                                                              │
│                                                                                                                 │
│  Task: Search the web for recent articles, reports, and discussions about AI Trends in 2025. Return only raw    │
│  facts, headlines, and data.                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Researcher                                                                                              │
│                                                                                                                 │
│  Thought: Action: Google Search                                                                                 │
│                                                                                                                 │
│  Using Tool: Google Search                                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  {                                                                                                              │
│    "search_query": "AI Trends 2025 articles reports discussions",                                               │
│    "location": "Worldwide"                                                                                      │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  An error occurred: 400 Client Error: Bad Request for url:                                                      │
│  https://serpapi.com/search?q=AI+Trends+2025+articles+reports+discussions&location=Worldwide&api_key=0fee5b107  │
│  ac76f2b3c2461a6149791bc8bceabb9041fd86b7a40de4f0f7a83d1. Some parameters may be invalid.                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Researcher                                                                                              │
│                                                                                                                 │
│  Thought: Thought: The previous search failed due to a bad request. I need to re-evaluate the search query and  │
│  potentially the location parameter. Since the task is to search for AI trends in 2025, a worldwide location    │
│  is appropriate, but the error message suggests "Some parameters may be invalid." I will try the same search    │
│  query without specifying a location, as the default might be sufficient and avoid the error.                   │
│                                                                                                                 │
│  Using Tool: Google Search                                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Analyst                                                                                                 │
│                                                                                                                 │
│  Task: Analyze the research data and identify the top 3 key trends or insights.                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Analyst                                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  1.  **AI is becoming a pervasive, strategic imperative across all sectors.**                                   │
│      AI is no longer a niche technology but a core organizational strategy, broadly investigated, applied, and  │
│  driving technological advancements across research, industry, geopolitics, and job transformation.             │
│  2.  **The emphasis has shifted to extracting real-world value and practical application from AI.**             │
│      Organizations are increasingly focused on how AI, particularly Generative AI, is being used to drive       │
│  tangible business value and solve complex problems, moving beyond theoretical potential to practical,          │
│  impactful usage.                                                                                               │
│  3.  **Despite widespread adoption, significant challenges and foundational infrastructure needs persist for    │
│  effective AI implementation.**                                                                                 │
│      While AI is strategically important, its successful adoption is hampered by common challenges,             │
│  necessitating robust foundational building blocks and strategic planning to support innovation and overcome    │
│  implementation hurdles.                                                                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Executive Writer                                                                                        │
│                                                                                                                 │
│  Task: Convert the insights into a one-page executive summary in markdown format.                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Executive Writer                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # Executive Briefing: AI - Strategic Imperative, Practical Value, and Implementation Challenges                │
│                                                                                                                 │
│  ## Overview                                                                                                    │
│  Artificial Intelligence (AI) has rapidly evolved from an emerging technology to a core strategic imperative    │
│  across all sectors. Organizations are now intensely focused on leveraging AI, particularly Generative AI, to   │
│  drive tangible business value. However, successful implementation necessitates addressing significant          │
│  foundational infrastructure needs and persistent challenges.                                                   │
│                                                                                                                 │
│  ## Key Insights                                                                                                │
│                                                                                                                 │
│  ### 1. AI as a Pervasive Strategic Imperative                                                                  │
│  AI is no longer a niche concern but a fundamental organizational strategy. Its influence spans research,       │
│  industry, geopolitics, and job market transformation, driving technological advancements and requiring broad   │
│  investigation and application across all facets of an enterprise. This widespread integration underscores      │
│  AI's critical role in future competitiveness and innovation.                                                   │
│                                                                                                                 │
│  ### 2. Shift to Real-World Value and Practical Application                                                     │
│  The emphasis has decisively moved from theoretical potential to practical, impactful usage. Organizations are  │
│  increasingly focused on how AI, especially Generative AI, can be applied to solve complex business problems    │
│  and deliver measurable value. This shift demands a clear strategy for identifying use cases that translate AI  │
│  capabilities into tangible outcomes and competitive advantages.                                                │
│                                                                                                                 │
│  ### 3. Persistent Challenges and Foundational Infrastructure Needs                                             │
│  Despite AI's strategic importance and widespread adoption efforts, significant hurdles remain for effective    │
│  implementation. These challenges include data quality, talent gaps, ethical considerations, and integration    │
│  complexities. Overcoming these requires robust foundational building blocks, strategic planning, and           │
│  sustained investment in core infrastructure to support innovation and ensure successful, scalable AI           │
│  deployment.                                                                                                    │
│                                                                                                                 │
│  ## Conclusion                                         

# Executive Briefing: AI - Strategic Imperative, Practical Value, and Implementation Challenges

## Overview
Artificial Intelligence (AI) has rapidly evolved from an emerging technology to a core strategic imperative across all sectors. Organizations are now intensely focused on leveraging AI, particularly Generative AI, to drive tangible business value. However, successful implementation necessitates addressing significant foundational infrastructure needs and persistent challenges.

## Key Insights

### 1. AI as a Pervasive Strategic Imperative
AI is no longer a niche concern but a fundamental organizational strategy. Its influence spans research, industry, geopolitics, and job market transformation, driving technological advancements and requiring broad investigation and application across all facets of an enterprise. This widespread integration underscores AI's critical role in future competitiveness and innovation.

### 2. Shift to Real-World Value and Practical Application
The em